In [1]:
import os
import random
import re
import math
from collections import OrderedDict

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from konlpy.tag import Mecab

In [2]:
USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
tagger = Mecab()

In [3]:
USE_CUDA

True

In [4]:
def pre_processing(file):
    with open(file, 'r', encoding='utf8') as f:
        lines = f.readlines()
    corpus = OrderedDict()
    title_i = -1
    group_id = -1
    line_id = 0
    for i, line in enumerate(lines):
        if line[:4] == '<doc':
            title_i = i + 1
            group_id += 1
            continue
        if i == title_i:
            continue
        for sline in line.splitlines():
            sline = re.sub(r'[^ ㄱ-ㅣ가-힣\.,]+', 'x', sline)
            sline = re.sub(r'다\.', '다.\n', sline)
            slines = sline.split('\n')
            for s in slines:
                s = s.strip()
                if len(s) > 20:
                    if 'x' not in s:
                        s = ' '.join(tagger.morphs(s))
                        corpus[line_id] = {}
                        corpus[line_id][group_id] = s
                        line_id += 1
    return corpus


def print_lines(corpus, n=10):
    for i in range(n):
        try:
            print(corpus[i])
        except ValueError:
            print('not enough lines')

In [5]:
corpus = pre_processing('wiki.txt')

In [6]:
print_lines(corpus)

{0: '지미 카터 는 조지 아주 섬터 카운티 플 레인스 마을 에서 태어났 다 .'}
{0: '조지아 주지사 로 지내 면서 , 미국 에 사 는 흑인 등 용법 을 내세웠 다 .'}
{0: '카터 대통령 은 에너지 개발 을 촉구 했으나 공화 당 의 반대 로 무산 되 었 다 .'}
{0: '카터 는 이집트 와 이스라엘 을 조정 하 여 , 캠프 데이비드 에서 안와르 사다트 대통령 과 메나헴 베긴 수상 과 함께 중동 평화 를 위한 캠프 데이비드 협정 을 체결 했 다 .'}
{0: '그러나 이것 은 공화 당 과 미국 의 유대 인 단체 의 반발 을 일으켰 다 .'}
{0: '지미 카터 는 대한민국 과 의 관계 에서 도 중요 한 영향 을 미쳤 던 대통령 중 하나 다 .'}
{0: '인권 문제 와 주한 미군 철수 문제 로 한때 한미 관계 가 불편 하 기 도 했 다 .'}
{0: '미국 의 빈곤층 지원 활동 , 사랑 의 집짓기 운동 , 국제 분쟁 중재 등 의 활동 도 했 다 .'}
{0: '카터 는 카터 행정부 이후 미국 이 북 핵 위기 , 코소보 전쟁 , 이라크 전쟁 과 같이 미국 이 군사 적 행동 을 최후 로 선택 하 는 전통 적 사고 를 버리 고 군사 적 행동 을 선행 하 는 행위 에 대해 깊 은 유감 을 표시 하 며 미국 의 군사 적 활동 에 강한 반대 입장 을 보이 고 있 다 .'}
{0: '특히 국제 분쟁 조정 을 위해 북한 의 김일성 , 아이 티 의 세드 라스 장군 , 팔레 인 스타인 의 하마스 , 보스니아 의 세르비 아계 정권 같이 미국 정부 에 대해 협상 을 거부 하 면서 사태 의 위기 를 초래 한 인물 및 단체 를 직접 만나 분쟁 의 원인 을 근본 적 으로 해결 하 기 위해 힘썼 다 .'}


In [15]:
with open('corpus.txt', 'w', encoding='utf8') as f:
    for values in corpus.values():
        for line in values.values():
            f.write(f'{line}\n')